In [85]:
import pandas as pd
import numpy as np
import math
import os
import json

import viewdb_to_bq  as x

import incident_data_pb2 as pb2_incident


from google.protobuf.timestamp_pb2 import Timestamp

from datetime import datetime ,timezone

# Parameter parameter

In [86]:
# file name
csv_file="incident_bq-storage-api.csv"

# get data from dict
view_name="xyz_incident"
view_name_id="incident_id"
datetimeCols=["open_datetime","close_datetime"]
pk_fkCols=['inventory_id', 'incident_id']

projectId="pongthorn"
main_dataset_id='SMartDataAnalytics'

table_name="incident"

dt_imported=datetime.now(timezone.utc) # utc
dt_imported=datetime.strptime(dt_imported.strftime("%Y-%m-%d %H:%M:%S"),"%Y-%m-%d %H:%M:%S")
dt_imported

# def example_function(**kwargs):
#     arg1 = kwargs.get('arg1', default_value1)
#     arg2 = kwargs.get('arg2', default_value2)
#     # ... process arguments

# # Usage
# example_function(arg1=value1, arg2=value2, arg3=value3, arg4=value4)


datetime.datetime(2024, 1, 17, 18, 48, 6)

# Init Const and Variable 

In [87]:
upsert_json_file="incident_upsert.json"
delete_json_file="incident_delete.json"

# BQ-Storage-API Data Transformation

In [88]:
df=pd.read_csv(csv_file)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   incident_id     2 non-null      int64  
 1   inventory_id    0 non-null      float64
 2   incident_type   0 non-null      float64
 3   service_type    0 non-null      float64
 4   severity        0 non-null      float64
 5   status          0 non-null      float64
 6   open_datetime   0 non-null      float64
 7   close_datetime  0 non-null      float64
 8   action          2 non-null      object 
dtypes: float64(7), int64(1), object(1)
memory usage: 272.0+ bytes


In [89]:
print("Add timestamp import")
dtimestamp = Timestamp()
dtimestamp.FromDatetime(dt_imported)
update_at_micro_timestampe =dtimestamp.ToMicroseconds()
df['update_at']=update_at_micro_timestampe 


Add timestamp import


In [90]:
print("Change action type")

def change_action_merge_to_bq_storage_api(x):
    if x=="added" or x=="changed":
        return  "UPSERT"
    else:
        return "DELETE"   
df["_CHANGE_TYPE"]=df['action'].apply(change_action_merge_to_bq_storage_api)
df=df.drop(columns=['action'])

Change action type


In [91]:
print("Make sure all pk and fk are Int64")
if len(pk_fkCols)>0:
 df[pk_fkCols] = df[pk_fkCols].astype('Int64')

Make sure all pk and fk are Int64


In [93]:
df.info()
df.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   incident_id     2 non-null      Int64  
 1   inventory_id    0 non-null      Int64  
 2   incident_type   0 non-null      float64
 3   service_type    0 non-null      float64
 4   severity        0 non-null      float64
 5   status          0 non-null      float64
 6   open_datetime   0 non-null      float64
 7   close_datetime  0 non-null      float64
 8   update_at       2 non-null      int64  
 9   _CHANGE_TYPE    2 non-null      object 
dtypes: Int64(2), float64(6), int64(1), object(1)
memory usage: 292.0+ bytes


,incident_id,inventory_id,incident_type,service_type,severity,status,open_datetime,close_datetime,update_at,_CHANGE_TYPE
0,4384,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,1705517286000000,DELETE
1,4329,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,1705517286000000,DELETE


# Spit data into UPSERT and DELETE 

In [94]:
dfUpsert=df.query("_CHANGE_TYPE=='UPSERT'")
dfUpsert.info()
dfUpsert.tail()


<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   incident_id     0 non-null      Int64  
 1   inventory_id    0 non-null      Int64  
 2   incident_type   0 non-null      float64
 3   service_type    0 non-null      float64
 4   severity        0 non-null      float64
 5   status          0 non-null      float64
 6   open_datetime   0 non-null      float64
 7   close_datetime  0 non-null      float64
 8   update_at       0 non-null      int64  
 9   _CHANGE_TYPE    0 non-null      object 
dtypes: Int64(2), float64(6), int64(1), object(1)
memory usage: 0.0+ bytes


,incident_id,inventory_id,incident_type,service_type,severity,status,open_datetime,close_datetime,update_at,_CHANGE_TYPE


In [95]:
dfDelete=df.query("_CHANGE_TYPE=='DELETE'")
dfDelete.info()
dfDelete

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 0 to 1
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   incident_id     2 non-null      Int64  
 1   inventory_id    0 non-null      Int64  
 2   incident_type   0 non-null      float64
 3   service_type    0 non-null      float64
 4   severity        0 non-null      float64
 5   status          0 non-null      float64
 6   open_datetime   0 non-null      float64
 7   close_datetime  0 non-null      float64
 8   update_at       2 non-null      int64  
 9   _CHANGE_TYPE    2 non-null      object 
dtypes: Int64(2), float64(6), int64(1), object(1)
memory usage: 180.0+ bytes


,incident_id,inventory_id,incident_type,service_type,severity,status,open_datetime,close_datetime,update_at,_CHANGE_TYPE
0,4384,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,1705517286000000,DELETE
1,4329,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,1705517286000000,DELETE


# Upsert dataframe tranformation

# Timezone and UTC Convert
## If you convert any time of any tz to timestampe for converting to Microseconds , it wll turn into UTC  so DateTime is UTC
* to_char((abc.incident_datetime AT TIME ZONE 'Asia/Bangkok'::text),
           'YYYY-MM-DD HH24:MI'::text)   AS open_datetime
* to_char((abc.incident_datetime AT TIME ZONE 'UTC'::text),
               'YYYY-MM-DD HH24:MI'::text)   AS open_datetime
* https://www.epochconverter.com/




## WorkAround Sol: null dattime is replaced with 0(GMT:1-1-1970 12:00:00 AM)

In [96]:
if dfUpsert.empty==False:
    print("Convert strng to datetime and microseconds accordingly")

    def convert_string_to_datetime_timestamp_microseconds (dt_str):
        if isinstance(dt_str, str):
            dt=datetime.strptime(dt_str,"%Y-%m-%d %H:%M")
            
            x_timestamp = Timestamp()
            x_timestamp.FromDatetime(dt)
            micro_x =x_timestamp.ToMicroseconds()
            
            return micro_x
        else:
            return np.nan
    #        
    
    for d in datetimeCols:
        #Check whick column contain null value if so, convert float64 to int 32"
        dfUpsert[d]=dfUpsert[d].apply(convert_string_to_datetime_timestamp_microseconds)
        dfUpsert[d]=dfUpsert[d].astype('Int64')
        
# #         # handle null datetime value(workaround) for 2.0 but it seem to support in 3.0
        # 1970-01-01 00:00:00 UTC
        dfUpsert[d] = dfUpsert[d].fillna(0)
        
dfUpsert.info()
dfUpsert

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   incident_id     0 non-null      Int64  
 1   inventory_id    0 non-null      Int64  
 2   incident_type   0 non-null      float64
 3   service_type    0 non-null      float64
 4   severity        0 non-null      float64
 5   status          0 non-null      float64
 6   open_datetime   0 non-null      float64
 7   close_datetime  0 non-null      float64
 8   update_at       0 non-null      int64  
 9   _CHANGE_TYPE    0 non-null      object 
dtypes: Int64(2), float64(6), int64(1), object(1)
memory usage: 0.0+ bytes


,incident_id,inventory_id,incident_type,service_type,severity,status,open_datetime,close_datetime,update_at,_CHANGE_TYPE


# Delete dataframe tranformation

In [97]:
dfDelete=dfDelete[[view_name_id,"_CHANGE_TYPE"]]
dfDelete.info()
dfDelete

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   incident_id   2 non-null      Int64 
 1   _CHANGE_TYPE  2 non-null      object
dtypes: Int64(1), object(1)
memory usage: 50.0+ bytes


,incident_id,_CHANGE_TYPE
0,4384,DELETE
1,4329,DELETE


# Write Json File to ingest to BQ by Buffer Protocol

In [98]:
if  dfUpsert.empty==False:
    
    print(upsert_json_file)
    json_file_path=os.path.join(upsert_json_file)

    json_incident_data = json.loads(dfUpsert.to_json(orient = 'records'))
    with open(upsert_json_file, "w") as outfile:
        json.dump(json_incident_data, outfile)

In [99]:
if  dfDelete.empty==False:
    
    print(delete_json_file)
    json_file_path=os.path.join(delete_json_file)
    json_incident_data = json.loads(dfDelete.to_json(orient = 'records'))
    with open(json_file_path, "w") as outfile:
        json.dump(json_incident_data, outfile)


incident_delete.json


# BufferProto to BQ 

In [100]:
def get_data_pb2(view_name):
    
    x_data_pb2=None
    if view_name == "xyz_incident": 
        x_data_pb2=pb2_incident.IncidentData()
    else:
        raise Exception("No specified view name to get data pb2")
        
    return x_data_pb2

In [101]:
listColumns= df.columns.tolist()
print(listColumns)

['incident_id', 'inventory_id', 'incident_type', 'service_type', 'severity', 'status', 'open_datetime', 'close_datetime', 'update_at', '_CHANGE_TYPE']


In [102]:
pb2_data=get_data_pb2(view_name)
print(pb2_data.DESCRIPTOR)

# Ingest data to Bigquery

In [103]:
if os.path.exists(upsert_json_file):
    print("OK-Upsert")
    x.write_json_to_bq(listColumns=listColumns,
                              x_data_pb2=pb2_data,
                              json_data_file=upsert_json_file,
                              project_name=projectId,
                              dataset_name=main_dataset_id,
                              table_name=table_name)
                              



In [104]:
if os.path.exists(delete_json_file):
    print("OK-Delete")
    x.write_json_to_bq(listColumns=listColumns,
                              x_data_pb2=pb2_data,
                              json_data_file=delete_json_file,
                              project_name=projectId,
                              dataset_name=main_dataset_id,
                              table_name=table_name)
                              


OK-Delete
Rows to table: 'projects/pongthorn/datasets/SMartDataAnalytics/tables/incident' have been written.


# Delete all processed files

In [105]:
print("Delete all files") 
listFilesToBeDeleted=[csv_file,upsert_json_file,delete_json_file]
for file in listFilesToBeDeleted:
    if os.path.exists(file):
        print(file)
        os.remove(file)


Delete all files
incident_bq-storage-api.csv
incident_delete.json


In [106]:
# return True